In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Ploting the individual cell traces

In [ ]:
import pynapple as nap
import scipy
onsets = np.load('StimOn.npy')
onsets=onsets.reshape(len(onsets),)
scan_time = np.load('CaTimeStamp.npy').reshape(71190,)
traces = np.load('CaRaw.npy')
sound = np.load('AudFreq.npy')
visual = np.load('VisDeg.npy')

ca_z=nap.TsdFrame(t=scan_time,d=scipy.stats.zscore(traces,axis=0))
ToneOn=onsets[0]

plot_int=nap.IntervalSet(start=[onsets[0]], end=[onsets[10]])
ca_plot=ca_z.restrict(plot_int)
fig, ax = plt.subplots(1,1, sharex=True,figsize=(20,5))
temp_cell=[0,1,2,7,8,9,10,12,13,14,15,17,20,21,22,24,25,26,77]
plt.plot(ca_plot.t,ca_plot[:,temp_cell]+np.arange(0,len(temp_cell))*6,color="k",linewidth=.8)
plt.xlabel('Timesteps', fontsize=14)
plt.ylabel('Cell ID', fontsize=14)

### Plotting the mean resonse PSTH of all frequencies

In [ ]:
closest_values = np.array([np.abs(scan_time - x).argmin() for x in onsets])
input_data = []
traces = traces.T
means = np.mean(traces, 1).reshape(traces.shape[0], 1)
stdevs = np.std(traces, 1).reshape(traces.shape[0], 1)
# z-scoring the events
traces = (traces - means) / stdevs
for i in closest_values:
            
    pre_event_slice = traces[:, i - 150:i]
    event_slice = traces[:, i].reshape(len(traces), 1)  
    post_event_slice = traces[:, i:i + 150]
            
    # Normalize the slices (remove the pre-event means)
    pre_event_means = np.mean(pre_event_slice, 1).reshape(pre_event_slice.shape[0], 1)
    post_event_slice -= pre_event_means
    pre_event_slice -= pre_event_means
            
    # Concatenate pre-event, event, and post-event slices
    concatenated_slice = np.concatenate([pre_event_slice, event_slice, post_event_slice], axis=1)  # Shape: (29, 121)
    input_data.append(concatenated_slice)
    
input_data=np.array(input_data)[:,:,90:]
print(input_data.shape)
t=np.arange(-2,5+1/30, 1/30)
plt.plot(t,np.mean(input_data, (0,1)))
plt.xlabel('Time')
plt.ylabel('Z-score')
plt.title('Mean response (PSTH) of all cells to all frequencies ')

### Ploting the mean response (PSTH) of all cells to all individual frequencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


unique_sounds = np.unique(sound)  # Assuming 'sound' is a numpy array
sound = np.array(sound).flatten()


# Create the subplots
fig, axes = plt.subplots(6, 1, figsize=(10, 6 * 2))
t=np.arange(-2,5+1/30, 1/30)
# Loop through the unique sounds and plot each one
for i, label in enumerate(unique_sounds):
    sound_input_data = input_data[sound == label]
    
    # Plot the mean across the specified dimensions (0,1)
    axes[i].plot(t,np.mean(sound_input_data, (0, 1)))
    
    # Set the title for each subplot
    axes[i].set_title(f"Frequency: {label}")

# Adjust the space between subplots
plt.subplots_adjust(hspace=0.5)  # Increase hspace for more space between subplots

# Add a single x-label for the entire figure
fig.text(0.5, 0.04, 'Time', ha='center', va='center', fontsize=14)

# Add a single y-label for the entire figure
fig.text(0.04, 0.5, 'Z- score', ha='center', va='center', rotation='vertical', fontsize=14)
fig.suptitle('Mean response (PSTH) of all cells to all individual frequencies')
# Display the plots
plt.show()


### Plotting cell response to a given frequency

In [ ]:
def get_cell_mean(data, cell_number, frequency):
    
    if frequency is None:
        filtered_data = data[:,cell_number,:]
        
        filtered_data  = np.mean(filtered_data, 0)
    else:
        filtered_data = data[sound == frequency]
       
        filtered_data = filtered_data[:,cell_number,:]
        filtered_data = np.mean(filtered_data, 0)
    return filtered_data

cell_number = 11
frequency = 12000
data = get_cell_mean(input_data, cell_number, frequency)
t=np.arange(-2, 5 + 1/30, 1/30)
plt.plot(t, data)
plt.xlabel('Time')
plt.ylabel('Z-score')
if frequency is None:
    plt.title(f'Mean response cell {cell_number}')
else:
    plt.title(f'Mean response cell {cell_number}, at frequency {frequency}')
plt.show()

### Plotting tuning curve of individual neurons

In [ ]:
unique_sounds = np.unique(sound)
sound=np.array(sound).flatten()
def get_tunning_curve(data, cell_number, sound_frequencies):
    tunning_curve_list=[]
    for frequency in sound_frequencies:
        filtered_data = data[sound == frequency]
            
        filtered_data = np.mean(filtered_data[:,cell_number,60:], (0,1))
        
        tunning_curve_list.append(filtered_data)
            
        
    return np.array(tunning_curve_list).T
    
cell_number = 40   
tunning_curve_list = get_tunning_curve(input_data,cell_number,unique_sounds)    
plt.plot(tunning_curve_list)
plt.xticks(ticks=np.arange(6), labels=unique_sounds, rotation=45)
plt.title(f'Tuning curve of cell {cell_number}')


plt.xlabel('Frequencies')
plt.ylabel('Z-score')    
plt.show()

### Tunning curve of all cells

In [ ]:
unique_sounds = np.unique(sound)
sound = np.array(sound).flatten()
def get_tunning_curve(data, sound_frequencies):
    tunning_curve_list=[]
    
    for frequency in sound_frequencies:
        filtered_data = data[sound == frequency]
        filtered_data = np.mean(filtered_data[:,:,60:],(0,1))
        tunning_curve_list.append(filtered_data)
    return np.array(tunning_curve_list).T
            
cell_number = None   
tunning_curve_list = get_tunning_curve(input_data, unique_sounds)
for tunning_curve in tunning_curve_list:
        
    plt.plot(tunning_curve)
    plt.xticks(ticks=np.arange(6), labels=unique_sounds, rotation=45)
    plt.title('Tunning curve of all cells')
plt.xlabel('Frequencies')
plt.ylabel('Z-score')    
plt.show()